In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from pandas.tseries.holiday import USFederalHolidayCalendar
import os
from datetime import datetime, time

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

print("Loading and preparing data...")

# --- 1. Load Data and Split by Time of Day ---
df = pd.read_csv("../data/clean/calls_by_district.csv")
df['CallDateTime'] = pd.to_datetime(df['CallDateTime'])
df = df[(df['CallDateTime'] >= '2017-01-01') & (df['CallDateTime'] < '2025-01-01')]

# Add time of day segment
df['hour'] = df['CallDateTime'].dt.hour
df['time_segment'] = 'DAY'  # Default
df.loc[(df['hour'] < 7) | (df['hour'] >= 19), 'time_segment'] = 'NIGHT'  # 7pm to 7am is NIGHT

# Group by date, district, and time segment
df['date'] = df['CallDateTime'].dt.date
daily_segments = df.groupby(['date', 'NAME', 'time_segment']).size().reset_index(name='call_volume')
daily_segments['date'] = pd.to_datetime(daily_segments['date'])

print(f"Initial data loaded: {len(daily_segments)} records")
print(f"Districts: {daily_segments['NAME'].nunique()}")
print(f"Time segments: {daily_segments['time_segment'].unique()}")

# --- Fill in missing combinations with zero values ---
all_dates = pd.date_range(start=daily_segments['date'].min(), end=daily_segments['date'].max(), freq='D')
all_districts = daily_segments['NAME'].unique()
all_time_segments = daily_segments['time_segment'].unique()

# Create a DataFrame with all possible combinations
complete_index = pd.MultiIndex.from_product(
    [all_dates, all_districts, all_time_segments],
    names=['date', 'NAME', 'time_segment']
)
complete_df = pd.DataFrame(index=complete_index).reset_index()

# Merge with original data, filling missing call volumes with 0
complete_df = complete_df.merge(
    daily_segments,
    on=['date', 'NAME', 'time_segment'],
    how='left'
)
complete_df['call_volume'] = complete_df['call_volume'].fillna(0)

# Replace original data with complete dataset
daily_segments = complete_df

print(f"Complete data created: {len(daily_segments)} records (including zero-call entries)")

# --- 2. Calculate city-level totals by date and time segment ---
city_totals = daily_segments.groupby(['date', 'time_segment'])['call_volume'].sum().reset_index()
city_totals.rename(columns={'call_volume': 'city_total_calls'}, inplace=True)

# Add city totals back to the district-level data
daily_segments = daily_segments.merge(
    city_totals,
    on=['date', 'time_segment'],
    how='left'
)

# Calculate the historical proportion of calls for each district by time segment
district_proportions = daily_segments.groupby(['NAME', 'time_segment'])['call_volume'].sum()
time_segment_totals = daily_segments.groupby(['time_segment'])['call_volume'].sum()

# Calculate proportions
for district in all_districts:
    for segment in all_time_segments:
        if (district, segment) in district_proportions.index:
            district_total = district_proportions[(district, segment)]
            segment_total = time_segment_totals[segment]
            if segment_total > 0:
                prop = district_total / segment_total
                daily_segments.loc[
                    (daily_segments['NAME'] == district) &
                    (daily_segments['time_segment'] == segment),
                    'historical_proportion'
                ] = prop
            else:
                daily_segments.loc[
                    (daily_segments['NAME'] == district) &
                    (daily_segments['time_segment'] == segment),
                    'historical_proportion'
                ] = 0
        else:
            daily_segments.loc[
                (daily_segments['NAME'] == district) &
                (daily_segments['time_segment'] == segment),
                'historical_proportion'
            ] = 0

# --- 3. Define Feature Engineering Function for City-Level Model ---
def add_city_features(df, reference_date, time_segment):
    """
    Add features to the dataframe for city-level prediction,
    only using data up to reference_date (exclusive) for the specific time segment.
    """
    # Get all city-level data up to but not including the reference date for this time segment
    historical_data = df[(df['date'] < reference_date) &
                         (df['time_segment'] == time_segment)].copy()

    # Get the data for the reference date and time segment
    prediction_day = df[(df['date'] == reference_date) &
                        (df['time_segment'] == time_segment)].copy()

    if prediction_day.empty:
        return pd.DataFrame()  # No data for this day/segment

    # Take the first row (they all have the same date features)
    prediction_day = prediction_day.iloc[[0]].copy()

    # Add time-based features
    prediction_day['day_of_week'] = prediction_day['date'].dt.dayofweek
    prediction_day['month'] = prediction_day['date'].dt.month
    prediction_day['day_of_month'] = prediction_day['date'].dt.day
    prediction_day['day_of_year'] = prediction_day['date'].dt.dayofyear
    prediction_day['week_of_year'] = prediction_day['date'].dt.isocalendar().week.astype(int)
    prediction_day['year'] = prediction_day['date'].dt.year

    # Cyclical encoding of day of year
    prediction_day['sin_doy'] = np.sin(2 * np.pi * prediction_day['day_of_year'] / 365.25)
    prediction_day['cos_doy'] = np.cos(2 * np.pi * prediction_day['day_of_year'] / 365.25)

    # Cyclical encoding of day of week
    prediction_day['sin_dow'] = np.sin(2 * np.pi * prediction_day['day_of_week'] / 7)
    prediction_day['cos_dow'] = np.cos(2 * np.pi * prediction_day['day_of_week'] / 7)

    # Is weekend/holiday
    prediction_day['is_weekend'] = prediction_day['day_of_week'].isin([5, 6]).astype(int)

    calendar = USFederalHolidayCalendar()
    holidays = calendar.holidays(start=prediction_day['date'].min(), end=prediction_day['date'].max())
    prediction_day['is_holiday'] = prediction_day['date'].isin(holidays).astype(int)

    # Sort historical data by date
    historical_data = historical_data.sort_values('date')

    # Skip if no historical data
    if len(historical_data) == 0:
        return pd.DataFrame()

    # Lag features for city totals (from most recent days)
    for lag in [1, 2, 3, 7, 14]:
        if len(historical_data) >= lag:
            lag_indices = historical_data.groupby('date')['city_total_calls'].mean().sort_index().tail(lag).index
            matching_rows = historical_data[historical_data['date'].isin(lag_indices)]

            if not matching_rows.empty:
                lag_value = matching_rows.iloc[-1]['city_total_calls']
                prediction_day[f'city_lag_{lag}'] = lag_value
            else:
                prediction_day[f'city_lag_{lag}'] = np.nan
        else:
            prediction_day[f'city_lag_{lag}'] = np.nan

    # Rolling averages for city totals
    city_history = historical_data.groupby('date')['city_total_calls'].mean().reset_index()
    city_history = city_history.sort_values('date')

    prediction_day['city_rolling_3d'] = city_history['city_total_calls'].tail(3).mean() if len(city_history) >= 3 else np.nan
    prediction_day['city_rolling_7d'] = city_history['city_total_calls'].tail(7).mean() if len(city_history) >= 7 else np.nan
    prediction_day['city_rolling_30d'] = city_history['city_total_calls'].tail(30).mean() if len(city_history) >= 30 else np.nan

    # Trend feature (3d avg - 7d avg)
    if len(city_history) >= 7:
        prediction_day['city_trend_3d'] = prediction_day['city_rolling_3d'] - prediction_day['city_rolling_7d']
    else:
        prediction_day['city_trend_3d'] = np.nan

    # Same day last week for city
    if len(city_history) >= 7:
        week_ago = reference_date - pd.Timedelta(days=7)
        week_ago_data = city_history[city_history['date'] == week_ago]

        if not week_ago_data.empty:
            prediction_day['city_same_day_last_week'] = week_ago_data['city_total_calls'].iloc[0]
        else:
            prediction_day['city_same_day_last_week'] = np.nan
    else:
        prediction_day['city_same_day_last_week'] = np.nan

    return prediction_day

# --- 4. Define Feature Engineering Function for District-Level Model ---
def add_district_features(df, reference_date, district, time_segment, city_prediction):
    """
    Add features to the dataframe for district-level prediction,
    including the city-level prediction as a feature.
    """
    # Get all data up to but not including the reference date for this district and time segment
    historical_data = df[(df['date'] < reference_date) &
                         (df['NAME'] == district) &
                         (df['time_segment'] == time_segment)].copy()

    # Get the data for the reference date, district, and time segment
    prediction_day = df[(df['date'] == reference_date) &
                        (df['NAME'] == district) &
                        (df['time_segment'] == time_segment)].copy()

    if prediction_day.empty:
        return pd.DataFrame()  # No data for this day/district/segment

    # Add city prediction and historical proportion as features
    prediction_day['city_prediction'] = city_prediction

    # Add estimated call volume based on historical proportion
    historical_prop = prediction_day['historical_proportion'].values[0]
    prediction_day['prop_estimate'] = historical_prop * city_prediction

    # Add time-based features
    prediction_day['day_of_week'] = prediction_day['date'].dt.dayofweek
    prediction_day['month'] = prediction_day['date'].dt.month
    prediction_day['day_of_month'] = prediction_day['date'].dt.day
    prediction_day['day_of_year'] = prediction_day['date'].dt.dayofyear
    prediction_day['week_of_year'] = prediction_day['date'].dt.isocalendar().week.astype(int)
    prediction_day['year'] = prediction_day['date'].dt.year

    # Cyclical encoding of day of year
    prediction_day['sin_doy'] = np.sin(2 * np.pi * prediction_day['day_of_year'] / 365.25)
    prediction_day['cos_doy'] = np.cos(2 * np.pi * prediction_day['day_of_year'] / 365.25)

    # Cyclical encoding of day of week
    prediction_day['sin_dow'] = np.sin(2 * np.pi * prediction_day['day_of_week'] / 7)
    prediction_day['cos_dow'] = np.cos(2 * np.pi * prediction_day['day_of_week'] / 7)

    # Is weekend/holiday
    prediction_day['is_weekend'] = prediction_day['day_of_week'].isin([5, 6]).astype(int)

    calendar = USFederalHolidayCalendar()
    holidays = calendar.holidays(start=prediction_day['date'].min(), end=prediction_day['date'].max())
    prediction_day['is_holiday'] = prediction_day['date'].isin(holidays).astype(int)

    # Sort historical data by date
    historical_data = historical_data.sort_values('date')

    # Skip if no historical data
    if len(historical_data) == 0:
        return pd.DataFrame()

    # Lag features (from most recent days)
    for lag in [1, 2, 3, 7, 14]:
        if len(historical_data) >= lag:
            lag_value = historical_data['call_volume'].iloc[-lag:].values[0]
            prediction_day[f'lag_{lag}'] = lag_value
        else:
            prediction_day[f'lag_{lag}'] = np.nan

    # Rolling averages
    prediction_day['rolling_3d'] = historical_data['call_volume'].iloc[-3:].mean() if len(historical_data) >= 3 else np.nan
    prediction_day['rolling_7d'] = historical_data['call_volume'].iloc[-7:].mean() if len(historical_data) >= 7 else np.nan
    prediction_day['rolling_30d'] = historical_data['call_volume'].iloc[-30:].mean() if len(historical_data) >= 30 else np.nan

    # Trend feature (3d avg - 7d avg)
    if len(historical_data) >= 7:
        prediction_day['trend_3d'] = prediction_day['rolling_3d'] - prediction_day['rolling_7d']
    else:
        prediction_day['trend_3d'] = np.nan

    # Same day last week
    if len(historical_data) >= 7:
        # Find dates 7 days ago
        week_ago = reference_date - pd.Timedelta(days=7)
        week_ago_data = historical_data[historical_data['date'] == week_ago]

        if not week_ago_data.empty:
            prediction_day['same_day_last_week'] = week_ago_data['call_volume'].iloc[0]
        else:
            prediction_day['same_day_last_week'] = np.nan
    else:
        prediction_day['same_day_last_week'] = np.nan

    # Calculate ratio of district calls to city total for recent history
    if len(historical_data) >= 14:
        recent_data = historical_data.iloc[-14:]
        district_sum = recent_data['call_volume'].sum()
        city_sum = recent_data['city_total_calls'].sum()
        if city_sum > 0:
            prediction_day['recent_proportion'] = district_sum / city_sum
        else:
            prediction_day['recent_proportion'] = 0
    else:
        prediction_day['recent_proportion'] = prediction_day['historical_proportion']

    # Create binary indicator for zero/non-zero (for potential hybrid model)
    prediction_day['has_calls'] = (prediction_day['call_volume'] > 0).astype(int)

    return prediction_day

# --- 5. Define features to use ---
# City-level model features
city_features = [
    'day_of_week', 'month', 'day_of_month', 'day_of_year', 'week_of_year', 'year',
    'sin_doy', 'cos_doy', 'sin_dow', 'cos_dow',
    'city_lag_1', 'city_lag_2', 'city_lag_3', 'city_lag_7', 'city_lag_14',
    'city_rolling_3d', 'city_rolling_7d', 'city_rolling_30d',
    'city_trend_3d', 'city_same_day_last_week',
    'is_weekend', 'is_holiday'
]

# District-level model features (including city prediction)
district_features = [
    'day_of_week', 'month', 'day_of_month', 'day_of_year', 'week_of_year',
    'sin_doy', 'cos_doy', 'sin_dow', 'cos_dow',
    'city_prediction', 'prop_estimate', 'historical_proportion', 'recent_proportion',
    'lag_1', 'lag_2', 'lag_3', 'lag_7',
    'rolling_3d', 'rolling_7d', 'rolling_30d',
    'trend_3d', 'same_day_last_week',
    'is_weekend', 'is_holiday'
]

# --- 6. Train/Test Split ---
train_dates = daily_segments[daily_segments['date'] < '2023-01-01']['date'].unique()
test_dates = daily_segments[daily_segments['date'] >= '2023-01-01']['date'].unique()

print(f"Training on data from {min(train_dates)} to {max(train_dates)}")
print(f"Testing on data from {min(test_dates)} to {max(test_dates)}")

# --- 7. Create models directory ---
os.makedirs('../models/hierarchical', exist_ok=True)

# --- 8. Build and train City-Level Model ---
print("\n=== Training City-Level Model ===")

# Dictionary to store city models
city_models = {}
city_scalers = {}

time_segments = daily_segments['time_segment'].unique()

for segment in time_segments:
    print(f"\nTraining city model for {segment} time segment")

    # --- 8.1 Create training data for city model ---
    city_train_data = []

    for date in sorted(train_dates):
        # Add features for this date/segment at city level
        date_features = add_city_features(daily_segments, date, segment)

        if not date_features.empty:
            # Calculate actual city total for this date
            city_total = daily_segments[
                (daily_segments['date'] == date) &
                (daily_segments['time_segment'] == segment)
                ]['city_total_calls'].mean()

            date_features['city_total_calls'] = city_total
            city_train_data.append(date_features)

    if not city_train_data:
        print(f"No training data for city model, {segment} segment, skipping")
        continue

    city_train_df = pd.concat(city_train_data)

    # Drop rows with NaN values
    city_train_df_clean = city_train_df.dropna(subset=city_features)

    if len(city_train_df_clean) < 30:  # Need at least 30 days of data
        print(f"Not enough clean training data for city model, {segment} segment ({len(city_train_df_clean)} records), skipping")
        continue

    print(f"City training data: {len(city_train_df_clean)} records")

    # --- 8.2 Prepare city training inputs and outputs ---
    X_city_train = city_train_df_clean[city_features]
    y_city_train = city_train_df_clean['city_total_calls']

    # --- 8.3 Scale city features ---
    city_scaler = StandardScaler()
    X_city_train_scaled = city_scaler.fit_transform(X_city_train)

    # Store scaler for later use
    city_scalers[segment] = city_scaler

    # --- 8.4 Build and train city model ---
    city_model = Sequential([
        Dense(64, activation='relu', input_shape=(X_city_train_scaled.shape[1],)),
        BatchNormalization(),
        Dropout(0.3),
        Dense(32, activation='relu'),
        BatchNormalization(),
        Dropout(0.2),
        Dense(16, activation='relu'),
        Dense(1, activation='relu')  # Using ReLU to ensure non-negative predictions
    ])

    city_model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss='mse',
        metrics=['mae']
    )

    early_stop = EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True,
        verbose=0
    )

    city_history = city_model.fit(
        X_city_train_scaled, y_city_train,
        validation_split=0.2,
        epochs=50,
        batch_size=16,
        callbacks=[early_stop],
        verbose=0
    )

    # Store model for later use
    city_models[segment] = city_model

    # Save city model
    city_model.save(f'../models/hierarchical/city_{segment}_model.h5')
    import joblib
    joblib.dump(city_scaler, f'../models/hierarchical/city_{segment}_scaler.pkl')

    print(f"City model for {segment} saved")

# --- 9. Build and train District-Level Models ---
districts = daily_segments['NAME'].unique()

# Store model metrics
model_metrics = []

# Track all test results
all_test_results = []

# Dictionary to hold city-level predictions
city_predictions = {}

# First, generate all city-level predictions for test dates
print("\n=== Generating City-Level Predictions for Test Dates ===")
for segment in time_segments:
    if segment not in city_models:
        print(f"No city model for {segment}, skipping")
        continue

    city_model = city_models[segment]
    city_scaler = city_scalers[segment]

    for date in sorted(test_dates):
        # Add features for this date/segment at city level
        date_features = add_city_features(daily_segments, date, segment)

        if date_features.empty:
            print(f"No features for city prediction on {date}, {segment}")
            continue

        # Handle missing values
        for feature in city_features:
            if date_features[feature].isna().any():
                # Fill with mean from training
                date_features[feature] = date_features[feature].fillna(0)

        # Predict city total
        X_city_test = date_features[city_features]
        X_city_test_scaled = city_scaler.transform(X_city_test)
        city_prediction = city_model.predict(X_city_test_scaled, verbose=0).flatten()[0]

        # Ensure non-negative integer prediction
        city_prediction = int(round(max(0, city_prediction)))

        # Store prediction
        city_predictions[(date, segment)] = city_prediction

        # For validation: get actual city total
        actual_city_total = daily_segments[
            (daily_segments['date'] == date) &
            (daily_segments['time_segment'] == segment)
            ]['city_total_calls'].mean()

        print(f"Date: {date}, Segment: {segment}, Predicted: {city_prediction:.1f}, Actual: {actual_city_total:.1f}")

print("\n=== Training District-Level Models ===")
for district in districts:
    for segment in time_segments:
        if segment not in city_models:
            print(f"No city model for {segment}, skipping district {district}")
            continue

        print(f"\n=== Training model for {district} - {segment} ===")

        # --- 9.1 Create training data for district model ---
        district_train_data = []

        for date in sorted(train_dates):
            # Get city prediction for this date (use actual for training)
            city_total = daily_segments[
                (daily_segments['date'] == date) &
                (daily_segments['time_segment'] == segment)
                ]['city_total_calls'].mean()

            # Add features for this date/district/segment
            date_features = add_district_features(daily_segments, date, district, segment, city_total)

            if not date_features.empty:
                district_train_data.append(date_features)

        if not district_train_data:
            print(f"No training data for {district} - {segment}, skipping")
            continue

        district_train_df = pd.concat(district_train_data)

        # Drop rows with NaN values
        district_train_df_clean = district_train_df.dropna(subset=district_features)

        if len(district_train_df_clean) < 30:  # Need at least 30 days of data
            print(f"Not enough clean training data for {district} - {segment} ({len(district_train_df_clean)} records), skipping")
            continue

        print(f"District training data: {len(district_train_df_clean)} records")

        # Check distribution of zero vs non-zero
        zero_pct = (district_train_df_clean['call_volume'] == 0).mean() * 100
        print(f"Zero percentage: {zero_pct:.1f}%")

        # --- 9.2 Prepare district training inputs and outputs ---
        X_district_train = district_train_df_clean[district_features]
        y_district_train = district_train_df_clean['call_volume']

        # --- 9.3 Scale district features ---
        district_scaler = StandardScaler()
        X_district_train_scaled = district_scaler.fit_transform(X_district_train)

        # --- 9.4 Build and train district model ---
        district_model = Sequential([
            Dense(64, activation='relu', input_shape=(X_district_train_scaled.shape[1],)),
            BatchNormalization(),
            Dropout(0.3),
            Dense(32, activation='relu'),
            BatchNormalization(),
            Dropout(0.2),
            Dense(16, activation='relu'),
            Dense(1, activation='relu')  # Using ReLU to ensure non-negative predictions
        ])

        district_model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
            loss='mse',
            metrics=['mae']
        )

        early_stop = EarlyStopping(
            monitor='val_loss',
            patience=10,
            restore_best_weights=True,
            verbose=0
        )

        # Use class weights for highly imbalanced data
        class_weights = None
        if zero_pct > 70:
            # Create weights for the effective "classes" even though this is regression
            # The idea is to weight samples differently based on their target value
            sample_weights = np.ones_like(y_district_train)
            sample_weights[y_district_train == 0] = 0.3  # Lower weight for zeros
            sample_weights[y_district_train > 0] = 1.0   # Normal weight for non-zeros
        else:
            sample_weights = None

        district_history = district_model.fit(
            X_district_train_scaled, y_district_train,
            validation_split=0.2,
            epochs=50,
            batch_size=16,
            callbacks=[early_stop],
            sample_weight=sample_weights,
            verbose=0
        )

        # --- 9.5 Evaluate on test data ---
        print("Evaluating model...")
        test_results = []

        for date in sorted(test_dates):
            # Get city prediction for this date from our pre-generated predictions
            if (date, segment) not in city_predictions:
                continue

            city_prediction = city_predictions[(date, segment)]

            # Add features for this date/district/segment
            date_features = add_district_features(daily_segments, date, district, segment, city_prediction)

            if date_features.empty:
                continue

            # Handle missing values
            for feature in district_features:
                if date_features[feature].isna().any():
                    # Fill with appropriate values or zeros
                    date_features[feature] = date_features[feature].fillna(0)

            # Predict
            X_district_test = date_features[district_features]
            X_district_test_scaled = district_scaler.transform(X_district_test)
            district_prediction = district_model.predict(X_district_test_scaled, verbose=0).flatten()[0]

            # Ensure non-negative integer prediction
            district_prediction = max(0, round(district_prediction))

            # Simple adjustment to ensure districts don't exceed city total
            # Only needed if we're running into issues with the sum of district predictions
            # exceeding the city prediction

            # Store results
            date_features['city_total_prediction'] = city_prediction
            date_features['district_prediction'] = district_prediction

            # Also store the naive proportion-based prediction for comparison
            date_features['proportion_prediction'] = round(date_features['prop_estimate'].values[0])

            test_results.append(date_features[['date', 'NAME', 'time_segment', 'call_volume',
                                               'city_total_calls', 'city_total_prediction',
                                               'historical_proportion', 'district_prediction',
                                               'proportion_prediction']])

        if not test_results:
            print(f"No test results for {district} - {segment}, skipping metrics")
            continue

        # Combine all test results
        test_results_df = pd.concat(test_results)

        # Add to global results
        all_test_results.append(test_results_df)

        # --- 9.6 Calculate Metrics ---
        # Hierarchical model metrics
        mae = mean_absolute_error(test_results_df['call_volume'], test_results_df['district_prediction'])
        rmse = np.sqrt(mean_squared_error(test_results_df['call_volume'], test_results_df['district_prediction']))

        # Calculate MAPE for non-zero actual values
        non_zero_mask = test_results_df['call_volume'] > 0
        if non_zero_mask.sum() > 0:
            mape = mean_absolute_percentage_error(
                test_results_df.loc[non_zero_mask, 'call_volume'],
                test_results_df.loc[non_zero_mask, 'district_prediction']
            ) * 100
        else:
            mape = np.nan

        # Proportion-based baseline metrics
        prop_mae = mean_absolute_error(test_results_df['call_volume'], test_results_df['proportion_prediction'])
        prop_rmse = np.sqrt(mean_squared_error(test_results_df['call_volume'], test_results_df['proportion_prediction']))

        print(f"Hierarchical Model - MAE: {mae:.2f}, RMSE: {rmse:.2f}, MAPE: {mape:.2f}%")
        print(f"Proportion Baseline - MAE: {prop_mae:.2f}, RMSE: {prop_rmse:.2f}")

        # Improvement over baseline
        mae_improvement = (prop_mae - mae) / prop_mae * 100
        print(f"MAE Improvement over proportion baseline: {mae_improvement:.1f}%")

        # Store metrics
        model_metrics.append({
            'district': district,
            'time_segment': segment,
            'mae': mae,
            'rmse': rmse,
            'mape': mape,
            'prop_mae': prop_mae,
            'prop_rmse': prop_rmse,
            'mae_improvement': mae_improvement,
            'avg_calls': test_results_df['call_volume'].mean(),
            'zero_pct': (test_results_df['call_volume'] == 0).mean() * 100
        })

        # --- 9.7 Save models ---
        model_path = f'../models/hierarchical/{district}_{segment}'
        district_model.save(f'{model_path}_district_model.h5')

        import joblib
        joblib.dump(district_scaler, f'{model_path}_district_scaler.pkl')

        print(f"District model and scaler saved to {model_path}")

        # --- 9.8 Visualize predictions ---
        plt.figure(figsize=(12, 6))
        plt.plot(test_results_df['date'], test_results_df['call_volume'], 'b-', label='Actual')
        plt.plot(test_results_df['date'], test_results_df['district_prediction'], 'r--', label='Hierarchical Model')
        plt.plot(test_results_df['date'], test_results_df['proportion_prediction'], 'g:', label='Proportion Baseline')
        plt.title(f"{district} - {segment} Call Volume Predictions")
        plt.xlabel('Date')
        plt.ylabel('Call Volume')
        plt.legend()
        plt.grid(alpha=0.3)

        # Add metrics to the plot
        plt.text(0.01, 0.95, f'Model MAE: {mae:.1f}, Baseline MAE: {prop_mae:.1f}',
                 transform=plt.gca().transAxes,
                 verticalalignment='top',
                 bbox=dict(boxstyle='round', facecolor='white', alpha=0.5))

        plt.savefig(f'../models/hierarchical/{district}_{segment}_predictions.png')
        plt.close()

# --- 10. Combine all test results ---
if all_test_results:
    combined_results = pd.concat(all_test_results)

    # --- 11. Overall Metrics ---
    overall_mae = mean_absolute_error(combined_results['call_volume'], combined_results['district_prediction'])
    overall_rmse = np.sqrt(mean_squared_error(combined_results['call_volume'], combined_results['district_prediction']))

    # Baseline metrics
    overall_prop_mae = mean_absolute_error(combined_results['call_volume'], combined_results['proportion_prediction'])
    overall_prop_rmse = np.sqrt(mean_squared_error(combined_results['call_volume'], combined_results['proportion_prediction']))

    # Calculate MAPE only for non-zero actuals
    non_zero_idx = combined_results['call_volume'] > 0
    if non_zero_idx.any():
        overall_mape = mean_absolute_percentage_error(
            combined_results.loc[non_zero_idx, 'call_volume'],
            combined_results.loc[non_zero_idx, 'district_prediction']
        ) * 100
    else:
        overall_mape = np.nan

    # Calculate improvement
    overall_improvement = (overall_prop_mae - overall_mae) / overall_prop_mae * 100

    print("\n===== Overall Performance =====")
    print(f"Hierarchical Model - MAE: {overall_mae:.2f}, RMSE: {overall_rmse:.2f}, MAPE: {overall_mape:.2f}%")
    print(f"Proportion Baseline - MAE: {overall_prop_mae:.2f}, RMSE: {overall_prop_rmse:.2f}")
    print(f"MAE Improvement: {overall_improvement:.1f}%")

    # --- 12. City-Level Prediction Accuracy ---
    city_results = combined_results.groupby(['date', 'time_segment']).agg({
        'city_total_calls': 'first',
        'city_total_prediction': 'first'
    }).reset_index()

    city_mae = mean_absolute_error(city_results['city_total_calls'], city_results['city_total_prediction'])
    city_rmse = np.sqrt(mean_squared_error(city_results['city_total_calls'], city_results['city_total_prediction']))

    print("\n===== City-Level Prediction Accuracy =====")
    print(f"MAE: {city_mae:.2f}, RMSE: {city_rmse:.2f}")

    # --- 13. Zero vs Non-Zero Performance ---
    zero_calls = combined_results[combined_results['call_volume'] == 0]
    non_zero_calls = combined_results[combined_results['call_volume'] > 0]

    if len(zero_calls) > 0:
        zero_mae = mean_absolute_error(zero_calls['call_volume'], zero_calls['district_prediction'])
        zero_prop_mae = mean_absolute_error(zero_calls['call_volume'], zero_calls['proportion_prediction'])

        print(f"\nPerformance on zero-call days:")
        print(f"Hierarchical Model - MAE: {zero_mae:.2f}")
        print(f"Proportion Baseline - MAE: {zero_prop_mae:.2f}")

        zero_correct = (zero_calls['district_prediction'] == 0).mean() * 100
        zero_prop_correct = (zero_calls['proportion_prediction'] == 0).mean() * 100

        print(f"Correctly predicted zeros - Model: {zero_correct:.1f}%, Baseline: {zero_prop_correct:.1f}%")

    if len(non_zero_calls) > 0:
        non_zero_mae = mean_absolute_error(non_zero_calls['call_volume'], non_zero_calls['district_prediction'])
        non_zero_prop_mae = mean_absolute_error(non_zero_calls['call_volume'], non_zero_calls['proportion_prediction'])

        print(f"\nPerformance on non-zero-call days:")
        print(f"Hierarchical Model - MAE: {non_zero_mae:.2f}")
        print(f"Proportion Baseline - MAE: {non_zero_prop_mae:.2f}")

    # --- 14. Model Metrics Table ---
    metrics_df = pd.DataFrame(model_metrics).sort_values('mae')
    print("\n===== Model Performance by District and Time Segment =====")
    print(metrics_df)

    # --- 15. Error Analysis ---
    errors = combined_results['call_volume'] - combined_results['district_prediction']
    baseline_errors = combined_results['call_volume'] - combined_results['proportion_prediction']

    # Plot error distributions side by side
    plt.figure(figsize=(12, 6))

    plt.subplot(1, 2, 1)
    plt.hist(errors, bins=30, alpha=0.7)
    plt.axvline(x=0, color='r', linestyle='--', alpha=0.7)
    plt.title('Hierarchical Model Error Distribution')
    plt.xlabel('Error (Actual - Predicted)')
    plt.ylabel('Frequency')
    plt.grid(alpha=0.3)

    plt.subplot(1, 2, 2)
    plt.hist(baseline_errors, bins=30, alpha=0.7)
    plt.axvline(x=0, color='r', linestyle='--', alpha=0.7)
    plt.title('Proportion Baseline Error Distribution')
    plt.xlabel('Error (Actual - Predicted)')
    plt.ylabel('Frequency')
    plt.grid(alpha=0.3)

    plt.tight_layout()
    plt.savefig('../models/hierarchical/error_distribution_comparison.png')
    plt.close()

    # --- 16. Compare with Original Neural Network ---
    try:
        nn_metrics = pd.read_csv('../models/district_segments/model_metrics.csv')
        print("\n===== Comparing Hierarchical and Original Neural Network Performance =====")

        # Create a comparative dataframe
        comparison = []

        for _, row in metrics_df.iterrows():
            district = row['district']
            segment = row['time_segment']

            # Find matching entry in NN metrics
            nn_row = nn_metrics[(nn_metrics['district'] == district) &
                                (nn_metrics['time_segment'] == segment)]

            if not nn_row.empty:
                comparison.append({
                    'district': district,
                    'time_segment': segment,
                    'hierarchical_mae': row['mae'],
                    'nn_mae': nn_row.iloc[0]['mae'],
                    'prop_mae': row['prop_mae'],
                    'vs_nn_diff': row['mae'] - nn_row.iloc[0]['mae'],
                    'vs_nn_pct': (row['mae'] - nn_row.iloc[0]['mae']) / nn_row.iloc[0]['mae'] * 100,
                    'vs_prop_pct': row['mae_improvement']
                })

        if comparison:
            comparison_df = pd.DataFrame(comparison).sort_values('vs_nn_pct')
            print(comparison_df)

            # Save comparison to CSV
            comparison_df.to_csv('../models/hierarchical/model_comparison.csv', index=False)
            print("\nComparison saved to ../models/hierarchical/model_comparison.csv")

            # Calculate average improvement
            avg_nn_improvement = comparison_df['vs_nn_pct'].mean()
            avg_prop_improvement = comparison_df['vs_prop_pct'].mean()

            print(f"\nAverage improvement vs NN: {avg_nn_improvement:.1f}%")
            print(f"Average improvement vs Proportion: {avg_prop_improvement:.1f}%")

            # Visualize the comparison
            plt.figure(figsize=(12, 8))

            # Sort districts by improvement
            best_districts = comparison_df.sort_values('vs_nn_pct').head(5)['district'].unique()

            # Prepare data for grouped bar chart
            district_segments = []
            hierarchical_maes = []
            nn_maes = []
            prop_maes = []

            for _, row in comparison_df[comparison_df['district'].isin(best_districts)].iterrows():
                district_segments.append(f"{row['district']}-{row['time_segment']}")
                hierarchical_maes.append(row['hierarchical_mae'])
                nn_maes.append(row['nn_mae'])
                prop_maes.append(row['prop_mae'])

            # Plot comparison
            x = np.arange(len(district_segments))
            width = 0.25

            plt.bar(x - width, hierarchical_maes, width, label='Hierarchical Model')
            plt.bar(x, nn_maes, width, label='Original NN')
            plt.bar(x + width, prop_maes, width, label='Proportion Baseline')

            plt.xlabel('District-Time Segment')
            plt.ylabel('MAE')
            plt.title('Model Comparison: MAE by District and Time Segment')
            plt.xticks(x, district_segments, rotation=45, ha='right')
            plt.legend()
            plt.tight_layout()
            plt.savefig('../models/hierarchical/model_comparison.png')
            plt.close()

    except Exception as e:
        print(f"\nCould not compare with original neural network results: {e}")

    # Save metrics to CSV
    metrics_df.to_csv('../models/hierarchical/model_metrics.csv', index=False)
    print("\nMetrics saved to ../models/hierarchical/model_metrics.csv")

    # --- 17. Save city-level predictions for future use ---
    city_predictions_df = city_results.copy()
    city_predictions_df.to_csv('../models/hierarchical/city_predictions.csv', index=False)
    print("\nCity predictions saved to ../models/hierarchical/city_predictions.csv")

    print("\nComplete! Created and evaluated hierarchical models.")
else:
    print("No test results generated. Check your data and model training process.")

Loading and preparing data...
Initial data loaded: 17984 records
Districts: 4
Time segments: ['DAY' 'NIGHT']
Complete data created: 23376 records (including zero-call entries)
Training on data from 2017-01-01 00:00:00 to 2022-12-31 00:00:00
Testing on data from 2023-01-01 00:00:00 to 2024-12-31 00:00:00

=== Training City-Level Model ===

Training city model for DAY time segment
City training data: 2161 records


City model for DAY saved

Training city model for NIGHT time segment
City training data: 2161 records


City model for NIGHT saved

=== Generating City-Level Predictions for Test Dates ===
Date: 2023-01-01 00:00:00, Segment: DAY, Predicted: 7.0, Actual: 14.0
Date: 2023-01-02 00:00:00, Segment: DAY, Predicted: 9.0, Actual: 7.0
Date: 2023-01-03 00:00:00, Segment: DAY, Predicted: 10.0, Actual: 6.0
Date: 2023-01-04 00:00:00, Segment: DAY, Predicted: 10.0, Actual: 14.0
Date: 2023-01-05 00:00:00, Segment: DAY, Predicted: 10.0, Actual: 11.0
Date: 2023-01-06 00:00:00, Segment: DAY, Predicted: 10.0, Actual: 15.0
Date: 2023-01-07 00:00:00, Segment: DAY, Predicted: 9.0, Actual: 10.0
Date: 2023-01-08 00:00:00, Segment: DAY, Predicted: 7.0, Actual: 9.0
Date: 2023-01-09 00:00:00, Segment: DAY, Predicted: 10.0, Actual: 3.0
Date: 2023-01-10 00:00:00, Segment: DAY, Predicted: 9.0, Actual: 4.0
Date: 2023-01-11 00:00:00, Segment: DAY, Predicted: 10.0, Actual: 7.0
Date: 2023-01-12 00:00:00, Segment: DAY, Predicted: 10.0, Actual: 5.0
Date: 2023-01-13 00:00:00, Segment: DAY, Predicted: 10.0, Actual: 13.0
Date

Hierarchical Model - MAE: 1.26, RMSE: 1.62, MAPE: 56.29%
Proportion Baseline - MAE: 1.24, RMSE: 1.60
MAE Improvement over proportion baseline: -1.5%
District model and scaler saved to ../models/hierarchical/FRIENDSHIP_DAY

=== Training model for FRIENDSHIP - NIGHT ===
District training data: 2161 records
Zero percentage: 18.7%
Evaluating model...


Hierarchical Model - MAE: 1.05, RMSE: 1.47, MAPE: 41.25%
Proportion Baseline - MAE: 1.06, RMSE: 1.46
MAE Improvement over proportion baseline: 0.6%
District model and scaler saved to ../models/hierarchical/FRIENDSHIP_NIGHT

=== Training model for ROUSS - DAY ===
District training data: 2161 records
Zero percentage: 22.1%
Evaluating model...


Hierarchical Model - MAE: 1.09, RMSE: 1.49, MAPE: 41.72%
Proportion Baseline - MAE: 1.06, RMSE: 1.44
MAE Improvement over proportion baseline: -3.2%
District model and scaler saved to ../models/hierarchical/ROUSS_DAY

=== Training model for ROUSS - NIGHT ===
District training data: 2161 records
Zero percentage: 61.0%
Evaluating model...


Hierarchical Model - MAE: 0.64, RMSE: 0.93, MAPE: 58.91%
Proportion Baseline - MAE: 0.69, RMSE: 0.95
MAE Improvement over proportion baseline: 6.5%
District model and scaler saved to ../models/hierarchical/ROUSS_NIGHT

=== Training model for SHAWNEE - DAY ===


KeyboardInterrupt: 